# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [12]:
%load_ext dotenv
%dotenv 

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [35]:
import dask

# Set the scheduler to use threads
dask.config.set(scheduler='threads')

import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [41]:
import os
from glob import glob
import dask.dataframe as dd

# Get the environment variable PRICE_DATA which should point to the directory containing parquet files
PRICE_DATA = os.getenv("PRICE_DATA")

# Use glob to find all parquet files within PRICE_DATA directory and its subdirectories
price_files = glob(PRICE_DATA+"/**/*.parquet/*.parquet")

# Read the parquet files into a Dask dataframe
price_df = dd.read_parquet(price_files)

# Display the first 42 rows of the dataframe
price_df.head(42)

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.532215,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.206837,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.712811,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.276840,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.299919,2819626,Health Care,Life Sciences Tools & Services,2000
A,2000-01-10,49.356224,49.803291,48.327969,49.311516,41.680588,2148446,Health Care,Life Sciences Tools & Services,2000
A,2000-01-11,49.311516,49.311516,47.523247,48.640915,41.113762,1855985,Health Care,Life Sciences Tools & Services,2000
A,2000-01-12,48.640915,48.640915,45.824390,47.657368,40.282406,1429874,Health Care,Life Sciences Tools & Services,2000
A,2000-01-13,48.909157,49.937412,47.210300,48.372677,40.887028,1134337,Health Care,Life Sciences Tools & Services,2000


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [38]:
# Calculate lagged 'Close' values and returns
dd_feat = (price_df.groupby('ticker', group_keys=False).apply(
    lambda df: df.assign(
        Close_lag_1 = df['Close'].shift(1),
        hi_lo_range = df['High'] - df['Low']
    )
)
# Calculate returns based on lagged Close price
.assign(
    returns = lambda df: (df['Close'] / df['Close_lag_1']) - 1
))
# Persist the Dask dataframe to improve performance
dd_feat = dd_feat.persist()

C:\Users\omovc\AppData\Local\Temp\ipykernel_5000\1383331095.py:3: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (price_df.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [39]:
# Write your code below.

# Convert Dask DataFrame to Pandas DataFrame
dd_feat = dd_feat.compute()

# Calculate rolling average return with 10-day window
grouped_data = dd_feat.groupby('ticker')

# Define a function to calculate rolling returns
def add_rolling_return(group):
    min_window = min(10, len(group))
    group['rolling_return_10d'] = group['returns'].rolling(min_window).mean()
    return group

# Apply the rolling return function to each group
dd_feat = grouped_data.apply(add_rolling_return).reset_index(level=1, drop=True)



Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

No, it was not necessary. Dask DataFrames support many pandas operations, including calculating moving averages, directly on Dask objects. This means you can compute moving averages efficiently using Dask without needing to convert your data to a pandas DataFrame first.

+ Would it have been better to do it in Dask? Why?

Yes, it would have been better to calculate the moving average return in Dask, especially for handling large datasets such as stock market data. Dask leverages parallel computing, optimizing performance, and allows for handling datasets that are larger-than-memory, calculating moving averages efficiently.

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [X] Created a branch with the correct naming convention.
- [X] Ensured that the repository is public.
- [X] Reviewed the PR description guidelines and adhered to them.
- [X] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.